In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train.csv')
train_df.info()

In [ ]:
train_df.head()

In [ ]:
train_df['target'].value_counts(normalize = True)

In [ ]:
train_df.describe()

In [ ]:
train_df.isna().sum().sort_values()

In [ ]:
plt.figure(figsize= (15,12))
sns.heatmap(train_df.corr())

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.info()

In [ ]:
test_df.describe()

In [ ]:
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier 
import lightgbm as lgb

In [ ]:
X = train_df.drop(columns= ['ID_code', 'target'])
y = train_df['target']

test= test_df.drop(columns = 'ID_code')

In [ ]:
fold=KFold(n_splits=5,shuffle=True,random_state=42)

oof=np.zeros(X.shape[0])
predictions=np.zeros(test.shape[0])

feature_importance_df=pd.DataFrame()
param = {
        'num_leaves': 10,
        'max_bin': 119,
        'min_data_in_leaf': 11,
        'learning_rate': 0.02,
        'min_sum_hessian_in_leaf': 0.00245,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': 14,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [ ]:
for fold_, (train_idx, val_idx) in enumerate(fold.split(X.values, y.values)):
    print("fold n{}".format(fold_))
    train_data = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx])
    val_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, train_data, num_round, valid_sets = [train_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(X.iloc[val_idx], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = X.columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test, num_iteration=clf.best_iteration) / 5

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof))) 

In [ ]:
submit = pd.DataFrame({'ID_code': test_df['ID_code'], 'target': predictions})

In [ ]:
submit.to_csv('submission.csv', index= False)